# 🎤 Whisper Realtime STT - Google Colab

**Ücretsiz, gerçek zamanlı konuşma tanıma**

- ✅ %100 Ücretsiz
- ✅ GPU Destekli (~1-2 saniye latency)
- ✅ Türkçe desteği mükemmel
- ✅ Public URL ile her yerden erişim

---

## 📋 Kurulum Adımları:

1. **Runtime > Change runtime type > GPU** seçin
2. Aşağıdaki hücreleri sırayla çalıştırın
3. Public URL'ye tıklayın ve kullanmaya başlayın!

---

## 1️⃣ Proje Dosyalarını Yükle

GitHub'dan projeyi klonlayın veya dosyaları manuel yükleyin.

In [ ]:
# Seçenek 1: GitHub'dan klonla (önerilir)
!git clone https://github.com/KULLANICI_ADI/whisperRealTime.git
%cd whisperRealTime

# Seçenek 2: Manuel yükleme
# Sol panelden "Files" > Upload ile dosyaları yükleyin:
# - colab_launcher.py
# - app.py
# - requirements.txt
# - templates/index.html

## 2️⃣ Gerekli Paketleri Kur

Bu işlem 2-3 dakika sürebilir.

In [ ]:
%%capture
!pip install flask flask-cors flask-sock faster-whisper numpy torch ctranslate2 -q

## 3️⃣ Cloudflare Tunnel Kur

Public URL için Cloudflare Tunnel kullanacağız.

In [ ]:
%%capture
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared
print("✅ Cloudflare Tunnel kuruldu!")

## 4️⃣ Uygulamayı Başlat

**ÖNEMLİ:** Bu hücre çalıştıktan sonra Public URL göreceksiniz. O linke tıklayın!

In [ ]:
# Launcher'ı import et ve başlat
import colab_launcher

# GPU ile başlat (önerilir)
colab_launcher.quick_start_gpu()

# CPU ile başlatmak isterseniz:
# colab_launcher.quick_start_cpu()

## ℹ️ Kullanım Notları

### Özellikler:
- **Model:** Whisper Small (dengeli performans)
- **Dil:** Türkçe (arayüzden değiştirilebilir)
- **Latency:** ~1-2 saniye (GPU ile)
- **VAD:** Otomatik sessizlik filtreleme

### Sorun Giderme:

**Public URL görünmüyorsa:**
- Hücreyi tekrar çalıştırın
- Runtime > Restart runtime yapıp baştan deneyin

**Mikrofon çalışmıyorsa:**
- Tarayıcıda mikrofon izni verin
- HTTPS bağlantısı gereklidir (Cloudflare otomatik sağlar)

**GPU kullanılmıyorsa:**
- Runtime > Change runtime type > GPU seçin
- Runtime'ı yeniden başlatın

### Performans İpuçları:

- **Tiny model** (~75MB): En hızlı, düşük kalite
- **Small model** (~500MB): Dengeli (önerilir)
- **Large-v3** (~3GB): En iyi kalite, yavaş

Model değiştirmek için `app.py` içinde `WHISPER_MODEL` değişkenini düzenleyin.

---

### 🎉 Tadını Çıkarın!

Public URL'nizi paylaşabilirsiniz. Link, Colab session açık kaldığı sürece çalışır.


## 🔍 URL Göremiyorsanız (Troubleshooting)

Eğer Public URL görünmüyor veya "..." ile kesikse bu hücreyi çalıştırın:

In [ ]:
# Cloudflared process'ini kontrol et ve URL'yi bul
import subprocess
import re

print("🔍 Cloudflare Tunnel URL'sini arıyorum...\n")

# Yöntem 1: Process listesinden bul
try:
    result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
    for line in result.stdout.split('\n'):
        if 'cloudflared' in line and 'tunnel' in line:
            print(f"✅ Cloudflare Tunnel çalışıyor\n")
            break
except:
    pass

# Yöntem 2: Yeni tunnel başlat ve URL'yi yakala
print("📡 URL'yi yakalamaya çalışıyorum...\n")
proc = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

url_found = False
for i in range(30):
    line = proc.stdout.readline()
    if line:
        print(line.strip())
        if 'trycloudflare.com' in line:
            url_match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
            if url_match:
                url = url_match.group(0)
                print("\n" + "="*70)
                print("✅ PUBLIC URL BULUNDU!")
                print("="*70)
                print(f"\n🌐 {url}\n")
                print("📝 Bu URL'yi kopyalayıp tarayıcınıza yapıştırın!")
                print("="*70)
                url_found = True
                break

if not url_found:
    print("\n⚠️ URL otomatik bulunamadı. Yukarıdaki çıktıda 'trycloudflare.com' içeren satırı arayın.")

## 🛑 Durdurmak İçin

Runtime > Interrupt execution veya aşağıdaki hücreyi çalıştırın:

In [ ]:
# Uygulamayı durdur
import os
import signal

# Tüm Python processlerini durdur
!pkill -f "python.*app.py"
!pkill -f cloudflared

print("🛑 Uygulama durduruldu.")